In [17]:
# Initialize Otter
import otter
grader = otter.Notebook("lab05.ipynb")

<img src="data6.png" style="width: 15%; float: right; padding: 1%; margin-right: 2%;"/>

# Lab 5 – Advanced Table Methods

## Data 6, Fall 2024

Today we will be exploring some more complex table methods we can use! The `apply` and `join` methods all allow us to perform different queries on our familiar tables. Understanding not only *how* each method works, but also *why* and *when* to use them are the key takeaways from this lab; by the end of it, we will be able to query tables in some pretty cool ways!

These new methods allow us to do different operations than before. As such, it is becoming more and more important to remember how each method sits on our Data Science toolbelt. We should think of each new method as a **tool that serves a specific purpose**. Your job as a data scientist is not only to understand what each tool does, but when each tool is applicable in new situations!

In [1]:
from datascience import *
import numpy as np

import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Part 1: Data Context Exploration

When working with data, it is always important to consider not only the impacts that the representation of the data can have in the real world</strong>, but also <strong>the effects that the conclusions from your data analysis and visualizations can have!</strong> 

### A Look Into the `Gender` Column

The [source of the original data](https://ir.ucr.edu/stats/enrollment/demographic) today from UC Riverside includes a dataset that displays admissions data, broken down by college. An interesting question you may want to consider is why UC Riverside would make this type of breakdown available alongside the original dataset of the whole university. 

Let's load the dataset into a Table object below called `ucr_college`.

In [2]:
ucr_college = Table.read_table("riverside_by_college.csv")
ucr_college.show(5)

Year,College,Level,Original Entry Type,New/Continuing Status,Full-Time Status,Gender,IPEDS Race/Ethnicity,Fall Headcount
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Asian,81
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Black/African American,5
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Chicano/Latino,78
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,International,10
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Two or More Races,7


As we can see from the `"Year"` column, this dataset also contains slightly more updated information, including data from 2023. Let's take a look at the way the data is represented in the `"Gender"` column for this dataset:

In [3]:
np.unique(ucr_college.column("Gender"))

array(['Female', 'Male', 'Nonbinary', 'Unknown'],
      dtype='<U9')

<!-- BEGIN QUESTION -->

### Question 1.1 (Discussion)
It looks like this dataset contains the additional values of `"Nonbinary"` and `"Unknown"` for `"Gender"`. Take a look at the "Details about the data" section [at the bottom of the website.](https://ir.ucr.edu/stats/enrollment/demographic) How was this particular variable measured? How might this impact our comparison between the conclusions we come to with this data vs. pre-2016 data?

By either looking at the details of the data or the ["Data Dictionary and Reporting Methodology"](https://ir.ucr.edu/sites/default/files/2019-07/IR_Data_Dictionary_1.pdf), find one other interesting way that UCR has chosen to measure their variables OR an interesting way UCR has chosen to capture concepts using certain variables and list it below.

*Hint:* Think about what reducing the number of categories would mean.

Pre 2016 Data:
Male: 20000
Female: 20000

Post 2016 Data:
Male: 18000
Female: 18000
Nonbinary: 3000
Unknown: 1000

This variable was measured using self-reported data in the UC admissions process. This might impact our comparison because we cannot use raw numbers for male and female students. We also can't do any analysis on nonbinary students pre-2016.

<!-- END QUESTION -->

Doing this type of extra basic background research on the source of our data can be incredibly helpful in our understanding of our data!

---
### Exploration of Aggregation by College

> "STEM Majors include those in the Bourns College of Engineering (BCOE) and the College of Natural and Agricultural Sciences (CNAS)."

Using this information, we can use the `where` method to create two different tables: one for STEM majors, and one for non-STEM majors. In the cell below, we generate the list of distinct college names at UC Riverside. As we see in the statement above, we can classify all non-STEM colleges as those in the list below, aside from the College of Engineering and the College of Natural and Agricultural Sciences.

In [4]:
np.unique(ucr_college.column("College"))

array(['Bourns College of Engineering',
       'College of Humanities, Arts, and Social Sciences',
       'College of Natural and Agricultural Sciences',
       'Graduate School of Education', 'School of Business',
       'School of Medicine', 'School of Public Policy'],
      dtype='<U48')

### Question 1.2
In the two cells below, use the `where` method to create two different tables: one for STEM majors (`ucr_stem`), and one for non-STEM majors (`ucr_nonstem`), based on which college they belong to.

In [20]:
ucr_stem = ucr_college.where("College", are.contained_in(make_array("Bourns College of Engineering", "College of Natural and Agricultural Sciences")))
ucr_stem.show(3)

Year,College,Level,Original Entry Type,New/Continuing Status,Full-Time Status,Gender,IPEDS Race/Ethnicity,Fall Headcount
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Asian,81
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Black/African American,5
2023,Bourns College of Engineering,Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Chicano/Latino,78


In [22]:
ucr_nonstem = ucr_college.where("College", are.not_contained_in(make_array("Bourns College of Engineering", "College of Natural and Agricultural Sciences")))
ucr_nonstem.show(3)

Year,College,Level,Original Entry Type,New/Continuing Status,Full-Time Status,Gender,IPEDS Race/Ethnicity,Fall Headcount
2023,"College of Humanities, Arts, and Social Sciences",Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Asian,486
2023,"College of Humanities, Arts, and Social Sciences",Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Black/African American,72
2023,"College of Humanities, Arts, and Social Sciences",Undergraduate,Undergraduate Frosh,New,Full-Time,Female,Chicano/Latino,867


In [23]:
grader.check("q1_2")

q1_2 results: All test cases passed!

Now, if we wanted to do a rough tabular comparison of the STEM vs. non-STEM majors, we can group by `"Gender"` and compare the numbers.

For now, don't worry too much about understanding the syntax for the group method: we'll cover it in the next lab! The code in the two cells below simply aggregates the data by calculating the sums of the `"Fall Headcount"` column for each gender category.

In [28]:
ucr_stem_grouped = ucr_stem.group("Gender", np.sum).select("Gender", "Fall Headcount sum").sort("Fall Headcount sum", descending=True)
ucr_stem_grouped 

Gender,Fall Headcount sum
Male,132516
Female,96494
Unknown,1063
Nonbinary,428


In [29]:
ucr_nonstem_grouped = ucr_nonstem.group("Gender", np.sum).select("Gender", "Fall Headcount sum").sort("Fall Headcount sum", descending=True)
ucr_nonstem_grouped 

Gender,Fall Headcount sum
Female,195432
Male,136041
Unknown,1499
Nonbinary,1166


<!-- BEGIN QUESTION -->

### Question 1.3 (Discussion)
From the above two tables, we can see an interesting discrepancy that we see quite often with this type of education-related data. What do you see when we compare these two tables?

We can see that generally, there are more male-identifying students than female-identifying students in STEM-majors. 
We can also see that the nonbinary students generally take on non-stem majors.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1.4
Below, create a joined version of the headcounts split by gender for with one column containing the fall headcounts for STEM colleges, and another containing the headcounts for non-STEM colleges.

To differentiate between STEM and non-STEM majors, label the STEM headcount "Fall STEM Headcount" and non-STEM headcount "Non-STEM Fall Headcount". You may find the `relabeled` table method useful here.

In [32]:
ucr_joined_gender = ucr_stem_grouped.join("Gender", ucr_nonstem_grouped).relabeled("Fall Headcount sum", "Fall STEM Headcount").relabeled("Fall Headcount sum_2", "Non-STEM Fall Headcount")
ucr_joined_gender

Gender,Fall STEM Headcount,Non-STEM Fall Headcount
Female,96494,195432
Male,132516,136041
Nonbinary,428,1166
Unknown,1063,1499


<!-- END QUESTION -->

We looked into some aspects of the students' race/ethnicities in Part 2, but now we can dig a little deeper by performing the same process of grouping our data and visualizing the distribution between the STEM and non-STEM colleges. We start by creating the tables split by STEM and non-STEM, as well as the joined version with the headcounts of both categories.

In [33]:
ucr_stem_grouped_re = ucr_stem.group("IPEDS Race/Ethnicity", np.sum).select(["IPEDS Race/Ethnicity", "Fall Headcount sum"]).sort("Fall Headcount sum", descending=True)
ucr_stem_grouped_re 

IPEDS Race/Ethnicity,Fall Headcount sum
Asian,89621
Chicano/Latino,54809
White,43765
International,18424
Domestic Unknown,8060
Black/African American,7889
Two or More Races,7356
American Indian/Alaskan Native,440
Native Hawaiian/Pacific Islander,137


In [34]:
ucr_nonstem_grouped_re = ucr_nonstem.group("IPEDS Race/Ethnicity", np.sum).select("IPEDS Race/Ethnicity", "Fall Headcount sum").sort("Fall Headcount sum", descending=True)
ucr_nonstem_grouped_re 

IPEDS Race/Ethnicity,Fall Headcount sum
Chicano/Latino,112421
Asian,101707
White,64222
Black/African American,18210
International,14528
Domestic Unknown,13140
Two or More Races,8466
American Indian/Alaskan Native,1082
Native Hawaiian/Pacific Islander,362


<!-- BEGIN QUESTION -->

### Question 1.5
Below, create a joined version of the headcounts split by IPEDS Race/Ethnicity for with one column containing the fall headcounts for STEM colleges, and another containing the headcounts for non-STEM colleges.

<!-- END QUESTION -->

Going back to an idea we posed when starting to look into the `"Gender"` column: while this type of exploration is very important and can also be informative, it is also important to remember that it is not always enough to simply look at a trend like this and state it. Ultimately, it is typically very hard to encode data about humans into numbers and categories, **because in doing so, we lose information and context about the individual we are looking at.** In your analysis in the future, try to strike a balance of looking at interesting trends in the data and considering the original context of the data you're working with.

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Part 2: Table Methods with Movies

In this section, we'll be working with a `movies` data set that contains information about various American films over time. It contains the following columns:
1. `"Film"`: The name of the movie
2. `"Genre"`: The genre of the movie
3. `"Year"`: The year the movie was released
4. `"Lead Studio"`: The primary movie studio responsible for producing the movie
5. `"Audience score %"`: The score, out of 100%, given to the movie by viewers
6. `"Rotten Tomatoes %"`: The score, out of 100%, given to the movies by the website [Rotten Tomatoes](https://www.rottentomatoes.com/)
7. `"Worldwide Gross (Millions)"`: The total gross revenue, in millions of dollars, that the movie made
8. `"Quality"`: Descriptive ranking of the movie based on audience score

Let's load the dataset into a new table, `movies`, to get practice with the more advanced table methods.

In [37]:
movies = Table.read_table("movies.csv")
movies.show(5)

Film,Genre,Year,Lead Studio,Audience score %,Rotten Tomatoes %,Worldwide Gross (Millions),Quality
You Will Meet a Tall Dark Stranger,Comedy,2010,Independent,35,43,26.66,Okay
When in Rome,Comedy,2010,Disney,44,15,43.04,Okay
What Happens in Vegas,Comedy,2008,Fox,72,28,219.37,Good
Water For Elephants,Drama,2011,20th Century Fox,72,60,117.09,Good
WALL-E,Animation,2008,Disney,89,96,521.28,Great


## The [apply](http://data8.org/datascience/_autosummary/datascience.tables.Table.apply.html#datascience.tables.Table.apply) method

The `apply` method allows us to map a function's behavior onto an entire column of a table. We can use built-in Python functions (like `max`) or we can define our own functions and then *apply* those functions to the columns of a table.

The `apply` method takes at least 2 arguments. The first is a function, and the rest are as many column labels you need to run that function. The number of columns you need to specify is dependent on the number of arguments the function has. For example, if the function you provide needs two inputs, you need to list two columns for it to work on.

`apply` returns a NumPy array of the transformed values. We can ask questions like "How can I categorize the items in this column?" (like converting grade percentages into letter grades from lecture). We can also make modifications to a table, like rounding all the values in a column to a certain accuracy.

Let's use `apply` to take the average of the two score percentages, `"Audience score %"` and `"Rotten Tomatoes %"`.

<!-- BEGIN QUESTION -->

### Question 2.1

In [38]:
def average_score(audience_score, rt_score):
    """Computes the average score between the audience score and Rotten Tomatoes score
    Inputs: 
    audience_score: an integer representing the audience score
    rt_score: an integer representing the Rotten Tomatoes score

    Returns:
    An integer representing the average of the two scores
    """
    return (audience_score + rt_score) / 2

In [39]:
average_scores = movies.apply(average_score, "Audience score %", "Rotten Tomatoes %")
average_scores

array([ 39. ,  29.5,  50. ,  66. ,  92.5,  78. ,  29.5,  35.5,  46.5,
        47. ,  65.5,  41. ,  52.5,  51.5,  58.5,  51.5,  35.5,  64. ,
        77. ,  33.5,  88.5,  31.5,  58.5,  49. ,  73. ,  63. ,  51.5,
        31. ,  31.5,  45.5,  50. ,  51.5,  70. ,  28. ,  66.5,  44. ,
        74. ,  88.5,  70. ,  64.5,  37. ,  29. ,  51.5,  45. ,  31.5,
        51. ,  35. ,  87. ,  28. ,  51.5,  81. ,  59.5,  64. ,  70.5,
        51. ,  32. ,  54.5,  54. ,  37. ,  39. ,  45.5,  86.5,  47.5,
        82. ,  69. ,  76.5,  84. ,  55.5,  84. ])

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2.2

Now, let's add a new column called `"Average score %"` and populate it with the information we just assigned to `average_scores`. We'll re-assign this new table to `movies`.

In [40]:
movies = movies.with_column("Average score %", average_scores)
movies.show(5)

Film,Genre,Year,Lead Studio,Audience score %,Rotten Tomatoes %,Worldwide Gross (Millions),Quality,Average score %
You Will Meet a Tall Dark Stranger,Comedy,2010,Independent,35,43,26.66,Okay,39
When in Rome,Comedy,2010,Disney,44,15,43.04,Okay,29.5
What Happens in Vegas,Comedy,2008,Fox,72,28,219.37,Good,50
Water For Elephants,Drama,2011,20th Century Fox,72,60,117.09,Good,66
WALL-E,Animation,2008,Disney,89,96,521.28,Great,92.5


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2.3

Explain what the error is with the code below:

In [41]:
movies.apply(average_score, "Genre", "Lead Studio")

TypeError: unsupported operand type(s) for /: 'str' and 'int'

This is telling us that we're trying to divide a string and an integer (which is not allowed).

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2.4
Fill in the function `convert_to_dollars` which converts a dollar amount from *millions of dollars* to *dollars*. Then, use the `apply` method to convert all values from the `"Worldwide Gross (Millions)"` column into dollars. Finally, create a new column in the `movies` table called `"Worldwide Gross"` using the array resulting from your call to `apply`.

*Note*: The code for this question requires several steps. Feel free to create new cells to experiment!


In [42]:
def convert_to_dollars(dollar_millions):
    """Converts a dollar amount from millions of dollars to dollars
    Inputs:
    dollar_millions: an integer representing the number of dollars in millions

    Returns:
    an integer representing the number of dollars
    """
    return dollar_millions * 1000000

dollars = movies.apply(convert_to_dollars, "Worldwide Gross (Millions)")
movies = movies.with_column("Worldwide Gross", dollars)
movies

Film,Genre,Year,Lead Studio,Audience score %,Rotten Tomatoes %,Worldwide Gross (Millions),Quality,Average score %,Worldwide Gross
You Will Meet a Tall Dark Stranger,Comedy,2010,Independent,35,43,26.66,Okay,39,2.666e+07
When in Rome,Comedy,2010,Disney,44,15,43.04,Okay,29.5,4.304e+07
What Happens in Vegas,Comedy,2008,Fox,72,28,219.37,Good,50,2.1937e+08
Water For Elephants,Drama,2011,20th Century Fox,72,60,117.09,Good,66,1.1709e+08
WALL-E,Animation,2008,Disney,89,96,521.28,Great,92.5,5.2128e+08
Waitress,Romance,2007,Independent,67,89,22.18,Good,78,2.218e+07
Waiting For Forever,Romance,2011,Independent,53,6,0.03,Okay,29.5,30000
Valentine's Day,Comedy,2010,Warner Bros.,54,17,217.57,Okay,35.5,2.1757e+08
Tyler Perry's Why Did I get Married,Romance,2007,Independent,47,46,55.86,Okay,46.5,5.586e+07
Twilight: Breaking Dawn,Romance,2011,Independent,68,26,702.17,Good,47,7.0217e+08


<!-- END QUESTION -->

## The [join](http://data8.org/datascience/_autosummary/datascience.tables.Table.join.html#datascience.tables.Table.join) method

The last method we will talk about in this lab is the `join` method. This method allows us to combine two different tables together!

The `join` method takes in 2 mandatory arguments and 1 optional argument:

| **Column Name** | **Description** |
| --- | --- |
| `column_label` | a column to use to join |
| `other` | another table |
| *Optional:* `other_label` | `other`'s label to join on (if not the same as `column_label`) |

If `other` has a label in common with the table you are joining with and this common label is the one you want to join on, then you do not need to use the optional argument. If you want to join on another column label or if neither table has a column label in common, then you can use the optional `other_label`.

The way join works takes some getting used to, so let's look at some examples of `join` at work!

We have the `dogs` and `owners` tables below, take a look at them a bit before we move on so you understand what data they contain:

In [43]:
dogs = Table().with_columns(
    "Name", np.array(["Shefran", "Hero", "Fluffy", "Doge"]),
    "Breed", np.array(["Bichon Frise", "Shih-poo", "Corgi", "Coin"]),
    "Owner", np.array(["Su Min", "Elizabeth", "Isabella", "Elizabeth"]),
)
dogs

Name,Breed,Owner
Shefran,Bichon Frise,Su Min
Hero,Shih-poo,Elizabeth
Fluffy,Corgi,Isabella
Doge,Coin,Elizabeth


In [44]:
owners = Table().with_columns(
    "Owner", np.array(["Su Min", "Kenneth", "Isabella", "Elizabeth"]),
    "Owner Age", np.array([6, 17, 3, 18])
)
owners

Owner,Owner Age
Su Min,6
Kenneth,17
Isabella,3
Elizabeth,18


As you can see, we have a column in common: `Owner`. Let's join these two tables together so that we can have all the doggy data in one place!

In [45]:
doggy_data = dogs.join("Owner", owners)
doggy_data

Owner,Name,Breed,Owner Age
Elizabeth,Hero,Shih-poo,18
Elizabeth,Doge,Coin,18
Isabella,Fluffy,Corgi,3
Su Min,Shefran,Bichon Frise,6


This table now has all of our information in one place, which makes using it easier!

Now let's take a look at a more common example, where the column labels being named differently can cause a problem. We will use the exact same `dogs` and `owners` tables, but we will change a column label on `owners`:

In [46]:
owners_new_label = owners.relabeled("Owner", "Name")

display(dogs, owners_new_label)

Name,Breed,Owner
Shefran,Bichon Frise,Su Min
Hero,Shih-poo,Elizabeth
Fluffy,Corgi,Isabella
Doge,Coin,Elizabeth


Name,Owner Age
Su Min,6
Kenneth,17
Isabella,3
Elizabeth,18


Now if we try to use the `join` method like we did last time, we run into an issue...

In [47]:
doggy_data = dogs.join("Owner", owners_new_label)
doggy_data

ValueError: label "Owner" not in labels ('Name', 'Owner Age')

Because the `owners` table does not have a column label called `Owner`, we may try to use the one column label they do have in common: `Name`...

In [48]:
doggy_data = dogs.join("Name", owners_new_label)
doggy_data

...but this doesn't appear to work either. This is not an error, there is simply no table outputted by this join call!

This `join` call does not do what we want, because the `dogs` `Name` corresponds to the **dog's** name, but the `owners` `Name` corresponds to the **Owner's** name! No dog and owner have any of the same names, so there is no data to output in this `join` call!

Instead, we have to make sure we join on the `Owner` column from `dogs` and the `Name` column from `owners`! We can do this using the third *optional* argument in the `join` method:

In [49]:
doggy_data = dogs.join("Owner", owners_new_label, "Name")
doggy_data

Owner,Name,Breed,Owner Age
Elizabeth,Hero,Shih-poo,18
Elizabeth,Doge,Coin,18
Isabella,Fluffy,Corgi,3
Su Min,Shefran,Bichon Frise,6


In this table, the `Name` column now refers to the name of the dog, and the `Owner` column corresponds to the name of the owner!

### Question 2.5 (A Slightly Different `Join`): 
The `join` method can also change the number of rows in its output. If there are multiple rows in one table that match with one row in the other, the `join` method will include rows for each of these matches in the output. Also, if there is a row in a table with no match in the other, there will be no row in the output that represents this row. Let's implement both these situations in practice and see how they work:

In [50]:
# Just run this cell
# This new dogs table has a new extra dogs
more_dogs = dogs.with_rows([["Clifford", "Big Red", "Sandra"], ["Doug", "Golden Retriever", "Russell"]])
more_dogs

Name,Breed,Owner
Shefran,Bichon Frise,Su Min
Hero,Shih-poo,Elizabeth
Fluffy,Corgi,Isabella
Doge,Coin,Elizabeth
Clifford,Big Red,Sandra
Doug,Golden Retriever,Russell


**Task**: Before we execute the join between these tables, we should be able to calculate how many rows should there be in the output. Assign the variable `more_dog_owner_rows` to the number of rows that should result from joining `more_dogs` with `owners_new_label`. Run the cell below to see them again for clarity:

In [51]:
display(more_dogs, owners_new_label)

Name,Breed,Owner
Shefran,Bichon Frise,Su Min
Hero,Shih-poo,Elizabeth
Fluffy,Corgi,Isabella
Doge,Coin,Elizabeth
Clifford,Big Red,Sandra
Doug,Golden Retriever,Russell


Name,Owner Age
Su Min,6
Kenneth,17
Isabella,3
Elizabeth,18


In [52]:
more_dog_owner_rows = 4
more_dog_owner_rows

4

In [53]:
grader.check("q2_5")

q2_5 results: All test cases passed!

### Confirm your answer

Remember, each row in the `more_dogs` table only gets a row in the output if it matches a row in `owners_new_label`

In [54]:
# If an owner in the table has 2 dogs, both dogs should appear in the output
# If a dog has no owner in the owners table, the dog does not appear in the output
complex_doggy_data = more_dogs.join("Owner", owners_new_label, "Name")
complex_doggy_data

Owner,Name,Breed,Owner Age
Elizabeth,Hero,Shih-poo,18
Elizabeth,Doge,Coin,18
Isabella,Fluffy,Corgi,3
Su Min,Shefran,Bichon Frise,6


In [55]:
# Total of 4 rows
complex_doggy_data.num_rows

4

**Finally, for reference, here is the link to the Data 6 Python Reference (our Python cheat-sheet) so you can review some of the methods we've used for tables in this lab!**

[Python Reference](http://data6.org/fa24/reference)

## Done! 😇

Good luck with the homework!

## Pets of Data 6
Kaalu is living it up! Good luck on HW2!

<img src="kaalu.jpg" width="40%" alt="Black dog chilling on the floor"/>

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)